## **Exploring Semantic Link**
Notebook with examples and explanations for basic functions using Semantic Link
<br> First step: Select 'Lakehouses' on the left, 'Remove All Lakehouses', then add a Lakehouse you have access to or create a new one.
<br><br>[Documentation and tutorials](https://learn.microsoft.com/en-us/fabric/data-science/semantic-link-overview)
<br><br>[Detailed SemPy Function listing](https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python)

In [ ]:
# Import the SemanticLink package and other dependencies
# SemanticLink is included in the default environment starting the with the Spark3.4 env, no need to pip install any more
import sempy.fabric as fabric
import pandas as pd


In [ ]:
# The dir command will retrieve a list of functions available
dir(fabric)

In [ ]:
# There is a handy help function built-in as well. Use the syntax 'help(fabric.[function name from above])
help(fabric.refresh_dataset)

### List functions - gather information on semantic model elements
Using the 'List' functions you can quickly compile information about your semantic models.

In [ ]:
# fabric.list_capacities()
fabric.list_datasets()
# fabric.list_workspaces()

In [ ]:
# List all tables in the specified data model
tables_df = fabric.list_tables("NW_Custom_Dataset",extended=True)
display(tables_df)

In [ ]:
# Prep for writing back to LH table - remove spaces in column names, add the semantic model name
tables_df.columns = tables_df.columns.str.replace(' ','')
tables_df['SemanticModelName'] = "NW_Custom_Dataset"
tables_df

In [ ]:
# List all columns in the specified data model
columns_df = fabric.list_columns("NW_Custom_Dataset",extended=True)
display(columns_df)

In [ ]:
# Prep for writing back to LH table - remove spaces in column names, add the semantic model name
columns_df.columns = columns_df.columns.str.replace(' ','')
columns_df['SemanticModelName'] = "NW_Custom_Dataset"
columns_df

In [ ]:
# List all measures in the specified data model
measures_df = fabric.list_measures("NW_Custom_Dataset")
display(measures_df)

In [ ]:
# Prep for writing back to LH table - remove spaces in column names, add the semantic model name
measures_df.columns = measures_df.columns.str.replace(' ','')
measures_df['SemanticModelName'] = "NW_Custom_Dataset"
measures_df

#### All this can be easily written back to tables in the LH to build out a semantic model/data dictionary as documentation:

In [ ]:
# I've had the best luck converting the pandas df to a spark df before writing to the LH. Using overwrite in this scenario, but you could also append (with a timestamp)
spark.createDataFrame(tables_df).write.mode('overwrite').option("mergeSchema", "true").saveAsTable("ModelTables")
spark.createDataFrame(columns_df).write.mode('overwrite').option("mergeSchema", "true").saveAsTable("ModelColumns")
spark.createDataFrame(measures_df).write.mode('overwrite').option("mergeSchema", "true").saveAsTable("ModelMeasures")

#### We can also take a look at relationship information

In [ ]:
from sempy.relationships import plot_relationship_metadata
from sempy.relationships import find_relationships
from sempy.fabric import list_relationship_violations
from sempy.dependencies import plot_dependency_metadata

In [ ]:
help(find_relationships)

In [ ]:
# List all relationships
relationships_df = fabric.list_relationships("NW_Custom_Dataset")
relationships_df

In [ ]:
# Visually plot relationships
plot_relationship_metadata(relationships_df)

### Extract data from the model
There are several ways to access the model data, either using the business logic built-in via measures or DAX, or by querying the model directly

In [ ]:
# Evaluate a measure from a dataset. Dataset, Measure, GroupBy column(s)
invoices_msr_df = (
    fabric.evaluate_measure(
        "NW_Custom_Dataset",
        "TotalInvoicedSales",
        groupby_columns=["NW_Invoices[ProductName]"]
        )
)
invoices_msr_df = invoices_msr_df.sort_values(["TotalInvoicedSales"],ascending=False)

display(invoices_msr_df)

In [ ]:
# Evaluate DAX directly
dax_example_df = fabric.evaluate_dax(dataset="NW_Custom_Dataset",
    dax_string=
    """
    EVALUATE 
    SUMMARIZECOLUMNS(
        'NW_Invoices'[ProductName],
        "TotalInvoicedSales", 'NW_Invoices'[TotalInvoicedSales]
    )"""
)
dax_example_df = dax_example_df.sort_values(["[TotalInvoicedSales]"],ascending=False)
display(dax_example_df)

In [ ]:
# Direct Query an attached Lakehouse (Update the Lakehouse in the table signifiers)
from pyspark.sql.functions import month, year, col, desc, asc
invoices_df = spark.sql("SELECT * FROM SixthGreatLake.NW_Invoices")
# display(invoices_df)

# Aggregate Invoices to a Monthly Level
invoices_agg_df = invoices_df.withColumn("ProductName",invoices_df.ProductName)\
.groupBy("ProductName")\
.sum("ExtendedPrice")\
# .orderBy("ExtendedPrice")
invoices_agg_df.sort(desc("sum(ExtendedPrice)")).show()
# display(invoices_agg_df)

### Data Science: Time-series forecasting with Prophet
Now let's grab some data and run a forecast

In [ ]:
# Direct Query an attached Lakehouse (Update the Lakehouse in the table signifiers)
from pyspark.sql.functions import month, year, col
invoices_df = spark.sql("SELECT * FROM SixthGreatLake.NW_Invoices JOIN SixthGreatLake.NW_Orders ON SixthGreatLake.NW_Invoices.OrderID = SixthGreatLake.NW_Orders.OrderID")

# Aggregate Invoices to a Monthly Level
invoices_agg_df = invoices_df.withColumn("Month", month("OrderDate"))\
.withColumn("Year",year("OrderDate"))\
.groupBy("Year","Month")\
.sum("ExtendedPrice")\
.orderBy("Year","Month")

display(invoices_agg_df)

In [ ]:
# Convert to Pandas df and drop last value (incomplete month)
invoices_agg_dfpd = invoices_agg_df.toPandas()
invoices_final_df = invoices_agg_dfpd.drop(22)
invoices_final_df

Now let's use the open-source time series forecasting tool Prophet to run a forecast
<br><br>[Prophet info](https://pypi.org/project/prophet/)

In [ ]:
!pip install prophet

from pyspark.sql import SparkSession
from pyspark.sql.functions import month, year, col
from prophet import Prophet

# Initialize Spark session
spark = SparkSession.builder.appName("Prophet Forecasting").getOrCreate()

# Prepare the data for Prophet
invoices_final_df['ds'] = pd.to_datetime(invoices_final_df[['Year', 'Month']].assign(DAY=1))
invoices_final_df['y'] = invoices_final_df['sum(ExtendedPrice)']

# Fit the Prophet model
model = Prophet(yearly_seasonality=True, weekly_seasonality=False,daily_seasonality=False)
model.fit(invoices_final_df[['ds', 'y']])

# Create a DataFrame for future predictions (e.g., next 12 months)
future = model.make_future_dataframe(periods=12, freq='M')

# Forecast
forecast = model.predict(future)

In [ ]:
# Plot the forecast from the previous step
model.plot(forecast);
model.plot_components(forecast);

### Access the REST APIs quickly and easily
No need to worry about authentication, scopes, etc. A user running a notebook can access whatever endpoints are available based on their role permissions
<br><br>[Power BI REST API Docs](https://learn.microsoft.com/en-us/rest/api/power-bi/)
<br><br>[Fabric REST API Docs](https://learn.microsoft.com/en-us/rest/api/fabric/articles/)

In [ ]:
import json 

# Initialize the PowerBI REST client
client = fabric.PowerBIRestClient()

In [ ]:
# Make a 'GET' request to the selected endpoint. Notebook user must have permissions to make API request
response = client.get('v1.0/myorg/groups/baa7413d-7c02-4a2c-b8c3-e8ec3fec2ddc/datasets')
response

In [ ]:
# Parse the response and normalize to a dataframe
data = json.loads(response.text)
datasets_df = pd.json_normalize(data['value'])
display(datasets_df)

In [ ]:
# Initialize the Fabric REST Client
client_fab = fabric.FabricRestClient()

In [ ]:
# Make a 'GET' request to the selected endpoint. Notebook user must have permissions to make API request
response = client_fab.get('v1/workspaces/baa7413d-7c02-4a2c-b8c3-e8ec3fec2ddc/items')
response

In [ ]:
# Parse the response and normalize to a dataframe
data = json.loads(response.text)
fabitems_df = pd.json_normalize(data['value'])
display(fabitems_df)

### Can also use other Python libraries
Such as powerbiclient to show a report in your notebook

In [ ]:
# Use the PowerBIRESTClient to display a report
reports_df = fabric.list_reports()
# reports_df # Show the list of reports
report_name = "BC_Prices"
report_id = reports_df[reports_df['Name'] == report_name]['Id'].iloc[0]
# report_id # show the report id selected

In [ ]:
from powerbiclient import Report
report = Report(group_id=None, report_id=report_id)
report